<a href="https://colab.research.google.com/github/psjdavid/psjdavid/blob/main/mk2_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#0.0722453828
import os
import pandas as pd
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision import datasets, models, transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.amp import autocast
from torch.cuda.amp import GradScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torchvision.models as models

# GPU 확인 및 설정
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU 모델: {torch.cuda.get_device_name(0)}")
    print(f"GPU 개수: {torch.cuda.device_count()}")
    # GPU 성능 최적화
    torch.backends.cudnn.benchmark = True
else:
    print("GPU를 사용할 수 없습니다. 런타임 유형을 GPU로 변경해주세요.")

CUDA 사용 가능: True
GPU 모델: Tesla T4
GPU 개수: 1


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 데이터 경로
base_path = '/content/drive/MyDrive/open'
train_path = os.path.join(base_path, 'train')
test_path = os.path.join(base_path, 'test')
test_csv_path = os.path.join(base_path, 'test.csv')

# 테스트 변환
test_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

Using device: cuda


In [ ]:
# 2. 테스트 데이터셋 클래스
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, image_paths, ids, transform=None):
        self.image_paths = image_paths
        self.ids = ids
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        try:
            image = Image.open(self.image_paths[idx]).convert('RGB')
            image = np.array(image)
            if self.transform:
                augmented = self.transform(image=image)
                image = augmented['image']
            return image, self.ids[idx]
        except Exception as e:
            print(f"Error loading image {self.image_paths[idx]}: {e}")
            # 오류 발생 시 빈 이미지 반환
            dummy_image = torch.zeros((3, 224, 224))
            return dummy_image, self.ids[idx]

In [ ]:
train_path = os.path.join(base_path, 'train')
import torchvision.transforms as transforms
#from torch.utils.data import datasets
train_dataset = datasets.ImageFolder(train_path, transform=transforms.ToTensor())
class_names = train_dataset.classes
num_classes = len(class_names)
print(f"Number of classes: {num_classes}")
print(f"Class names: {class_names}")

Number of classes: 7
Class names: ['Andesite', 'Basalt', 'Etc', 'Gneiss', 'Granite', 'Mud_Sandstone', 'Weathered_Rock']


In [ ]:
# 4. 모델 로드
model = models.mobilenet_v3_small(weights=None)
model.classifier[3] = nn.Linear(in_features=1024, out_features=num_classes)
model = model.to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/open/best_model.pth', map_location=device))
model.eval()
print("Best model loaded successfully.")

Best model loaded successfully.


In [ ]:
# 5. 배치 처리 함수
def process_batch(batch_df, batch_idx, save_path):
    print(f"Processing batch {batch_idx} with {len(batch_df)} images...")

    # 현재 배치의 이미지 경로 생성
    batch_image_paths = [os.path.join(test_path, f"{row['ID']}.jpg")
                         for _, row in batch_df.iterrows()]
    batch_ids = batch_df['ID'].tolist()

    # 현재 배치에 대한 데이터셋 및 로더 생성
    batch_dataset = TestDataset(batch_image_paths, batch_ids, transform=test_transform)
    batch_loader = DataLoader(
        batch_dataset,
        batch_size=32,
        shuffle=False,
        num_workers=0,  # 단일 프로세스로 처리
        pin_memory=True
    )

    # 현재 배치에 대한 예측 수행
    predictions = []
    ids = []
    probs_list = []

    with torch.no_grad():
        for images, batch_ids in tqdm(batch_loader, desc=f"Batch {batch_idx}"):
            images = images.to(device)
            try:
                with autocast(device_type='cuda'):
                    outputs = model(images)

                # 확률 계산
                probs = torch.nn.functional.softmax(outputs, dim=1)
                probs_list.extend(probs.cpu().numpy())

                # 예측 클래스
                _, predicted = torch.max(outputs, 1)
                predictions.extend(predicted.cpu().numpy())
                ids.extend(batch_ids)
            except Exception as e:
                print(f"Error in prediction: {e}")
                # 오류 발생 시 기본값 설정
                dummy_preds = [0] * len(batch_ids)
                dummy_probs = [[1.0/num_classes] * num_classes] * len(batch_ids)
                predictions.extend(dummy_preds)
                probs_list.extend(dummy_probs)
                ids.extend(batch_ids)

    # 결과 데이터프레임 생성
    pred_labels = [class_names[pred] for pred in predictions]
    result_df = pd.DataFrame({
        'ID': ids,
        'label': pred_labels
    })

    # 클래스별 확률 열 추가
    for i, cls in enumerate(class_names):
        result_df[f'prob_{cls}'] = [probs[i] for probs in probs_list]

    # 결과 저장
    result_path = os.path.join(save_path, f'predictions_batch_{batch_idx}.csv')
    result_df.to_csv(result_path, index=False)
    print(f"Batch {batch_idx} results saved to {result_path}")

    return result_df

In [ ]:
# 6. 메인 처리 함수
def process_test_data(batch_size=500):
    # 결과 저장 경로
    results_dir = os.path.join(base_path, 'batch_results')
    os.makedirs(results_dir, exist_ok=True)

    # 테스트 CSV 로드
    test_df = pd.read_csv(test_csv_path)
    print(f"Loaded test CSV with {len(test_df)} entries")

    # 배치 수 계산
    num_batches = (len(test_df) + batch_size - 1) // batch_size
    all_results = []

    # 각 배치 처리
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(test_df))
        batch_df = test_df.iloc[start_idx:end_idx]

        # 배치 처리 및 결과 저장
        batch_result = process_batch(batch_df, i, results_dir)
        all_results.append(batch_result)

        print(f"Completed batch {i+1}/{num_batches}")

    # 모든 결과 합치기
    if all_results:
        final_df = pd.concat(all_results, ignore_index=True)
        final_path = os.path.join(base_path, 'test_predictions_all.csv')
        final_df.to_csv(final_path, index=False)
        print(f"All predictions combined and saved to {final_path}")

        # 제출용 간소화 버전 (ID와 label만)
        submission_df = final_df[['ID', 'label']]
        submission_path = os.path.join(base_path, 'submission.csv')
        submission_df.to_csv(submission_path, index=False)
        print(f"Submission file saved to {submission_path}")
    else:
        print("No results were generated.")

In [ ]:
# 7. 실행

if __name__ == "__main__":
    # 배치 크기 설정 (필요에 따라 조정)
    process_test_data(batch_size=64)

Loaded test CSV with 95006 entries
Processing batch 0 with 64 images...


Batch 0: 100%|██████████| 2/2 [00:00<00:00,  4.57it/s]


Batch 0 results saved to /content/drive/MyDrive/open/batch_results/predictions_batch_0.csv
Completed batch 1/1485
Processing batch 1 with 64 images...


Batch 1: 100%|██████████| 2/2 [00:00<00:00,  6.93it/s]


Batch 1 results saved to /content/drive/MyDrive/open/batch_results/predictions_batch_1.csv
Completed batch 2/1485
Processing batch 2 with 64 images...


...

Batch 220: 100%|██████████| 2/2 [00:49<00:00, 24.98s/it]


Batch 220 results saved to /content/drive/MyDrive/open/batch_results/predictions_batch_220.csv
Completed batch 221/1485
Processing batch 221 with 64 images...


Batch 221: 100%|██████████| 2/2 [00:49<00:00, 24.86s/it]


Batch 221 results saved to /content/drive/MyDrive/open/batch_results/predictions_batch_221.csv
Completed batch 222/1485
Processing batch 222 with 64 images...


Batch 222:  50%|█████     | 1/2 [00:22<00:22, 22.99s/it]

In [ ]:
# 경로 수정
test_path = '/content/local_test'
test_csv_path = '/content/local_test/test.csv'

# 테스트 이미지 경로 업데이트
test_df = pd.read_csv(test_csv_path)
test_image_paths = [os.path.join(test_path, f"{row['ID']}.jpg") for _, row in test_df.iterrows()]

/bin/bash: line 1: /usr/bin/cp: Argument list too long


In [ ]:
# 모든 이미지를 미리 메모리에 로드
print("이미지 미리 로드 중...")
preloaded_images = []
for path in tqdm(test_image_paths[:subset_size]):
    try:
        img = Image.open(path).convert('RGB')
        img = np.array(img)
        preloaded_images.append(img)
    except Exception as e:
        print(f"오류: {path} - {e}")
        preloaded_images.append(np.zeros((224, 224, 3), dtype=np.uint8))

# 수정된 데이터셋 클래스
class PreloadedTestDataset(torch.utils.data.Dataset):
    def __init__(self, images, ids, transform=None, labels=None):
        self.images = images
        self.ids = ids
        self.transform = transform
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        id_ = self.ids[idx]
        if self.labels is not None:
            label = self.labels[idx]
            return image, id_, label
        return image, id_

# 미리 로드된 데이터셋 생성
preloaded_test_dataset = PreloadedTestDataset(
    preloaded_images,
    test_ids[:subset_size],
    transform=test_transform,
    labels=test_labels[:subset_size] if test_labels is not None else None
)

test_loader = DataLoader(
    preloaded_test_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

이미지 미리 로드 중...


NameError: name 'subset_size' is not defined

In [ ]:
# 테스트 데이터 로드
import pandas as pd
from PIL import Image
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

try:
    test_df = pd.read_csv(test_csv_path)
    test_image_paths = [os.path.join(test_path, f"{row['ID']}.jpg") for _, row in test_df.iterrows()]
    test_ids = test_df['ID'].tolist()
    # 라벨이 있는지 확인
    test_labels = test_df['label'].tolist() if 'label' in test_df.columns else None
except Exception as e:
    print(f"Error loading test CSV: {e}")
    raise

# 테스트용 Dataset
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, image_paths, ids, transform=None, labels=None):
        self.image_paths = image_paths
        self.ids = ids
        self.transform = transform
        self.labels = labels  # 라벨이 있으면 저장, 없으면 None

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        image = np.array(image)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        id_ = self.ids[idx]
        if self.labels is not None:
            label = self.labels[idx]
            return image, id_, label
        return image, id_

# 테스트 데이터셋 및 로더
test_dataset = TestDataset(test_image_paths, test_ids, transform=test_transform, labels=test_labels)
test_loader = DataLoader(
    test_dataset,
    batch_size=32,  # GPU에서는 더 큰 배치 사이즈
    shuffle=False,
    num_workers=2,  # Colab GPU에서는 2-4개 워커 권장
    pin_memory=True  # GPU 전송 최적화
)

# 모델 로드 (최적 모델)
model = models.mobilenet_v3_small(weights=None)
model.classifier[3] = nn.Linear(in_features=1024, out_features=num_classes)
model = model.to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/open/best_model.pth', map_location=device))
print("Best model loaded successfully.")

# 테스트 예측
model.eval()
predictions = []
ids = []
true_labels = [] if test_labels is not None else None
test_probs = []  # 클래스별 확률 저장

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Testing"):
        if test_labels is not None:
            images, batch_ids, labels = batch
            true_labels.extend(labels)
        else:
            images, batch_ids = batch
        images = images.to(device)

        # 혼합 정밀도 사용
        with autocast(device_type='cuda'):
          outputs = model(images)

        # 확률 계산
        probs = torch.nn.functional.softmax(outputs, dim=1)
        test_probs.extend(probs.cpu().numpy())

        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        ids.extend(batch_ids)

# 예측 라벨
pred_labels = [class_names[pred] for pred in predictions]

# 라벨이 있는 경우: 정확도 계산 및 결과 분석
if test_labels is not None:
    correct = sum(1 for true, pred in zip(true_labels, predictions) if true == pred)
    total = len(true_labels)
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

    # 혼동 행렬
    cm = confusion_matrix(true_labels, predictions)
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names)
    plt.title("Test Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.savefig("/content/drive/MyDrive/open/test_confusion_matrix.png")
    plt.show()

    # 클래스별 정확도
    print("\nClass-wise Accuracy:")
    for i, cls in enumerate(class_names):
        idx = np.array(true_labels) == i
        if np.sum(idx) > 0:
            class_acc = np.sum(np.array(predictions)[idx] == i) / np.sum(idx) * 100
            print(f"{cls}: {class_acc:.2f}%")

    # 오분류 샘플 분석
    misclassified = np.where(np.array(true_labels) != np.array(predictions))[0]
    print(f"\nNumber of misclassified samples: {len(misclassified)}")

    # 오분류 상위 5개 출력
    if len(misclassified) > 0:
        top_misclassified = sorted(misclassified,
                            key=lambda i: np.max(test_probs[i]) - test_probs[i][true_labels[i]],
                            reverse=True)[:5]

        print("\nTop 5 confident misclassifications:")
        for idx in top_misclassified:
            true_cls = class_names[true_labels[idx]]
            pred_cls = class_names[predictions[idx]]
            confidence = test_probs[idx][predictions[idx]] * 100
            print(f"ID: {ids[idx]}, True: {true_cls}, Predicted: {pred_cls}, Confidence: {confidence:.2f}%")

# 라벨이 없는 경우: 예측 결과 저장
else:
    # 예측 결과를 DataFrame으로 저장
    import pandas as pd
    result_df = pd.DataFrame({
        'ID': ids,
        'label': pred_labels
    })

    # 클래스별 확률 열 추가
    for i, cls in enumerate(class_names):
        result_df[f'prob_{cls}'] = [probs[i] for probs in test_probs]

    # CSV 파일로 저장
    result_df.to_csv('/content/drive/MyDrive/open/test_predictions.csv', index=False)
    print("Predictions saved to /content/drive/MyDrive/open/test_predictions.csv")

    # 불확실한 예측 (낮은 확률) 확인
    max_probs = [np.max(probs) for probs in test_probs]
    uncertain_idx = np.argsort(max_probs)[:10]  # 확률이 가장 낮은 10개

    print("\nMost uncertain predictions:")
    for idx in uncertain_idx:
        pred_cls = pred_labels[idx]
        confidence = max_probs[idx] * 100
        print(f"ID: {ids[idx]}, Predicted: {pred_cls}, Confidence: {confidence:.2f}%")

<ipython-input-17-d25c142c77e5>:56: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/drive/MyDrive/open/best_model.pth', map_location

Best model loaded successfully.


Testing:   0%|          | 0/2969 [00:00<?, ?it/s]


FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/worker.py", line 351, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "<ipython-input-17-d25c142c77e5>", line 31, in __getitem__
    image = Image.open(self.image_paths[idx]).convert('RGB')
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/PIL/Image.py", li

In [ ]:
import pandas as pd
import os
import glob

# 배치 결과 파일들이 있는 디렉토리
batch_results_dir = '/content/drive/MyDrive/open/batch_results'

# 모든 배치 결과 파일 찾기
batch_files = glob.glob(os.path.join(batch_results_dir, 'predictions_batch_*.csv'))
print(f"Found {len(batch_files)} batch result files")

# 모든 파일 통합
all_predictions = []
for file in batch_files:
    try:
        # 각 배치 파일 읽기
        df = pd.read_csv(file)
        # ID와 label 열만 선택
        df = df[['ID', 'label']]
        all_predictions.append(df)
        print(f"Loaded {file} with {len(df)} entries")
    except Exception as e:
        print(f"Error loading {file}: {e}")

# 모든 데이터프레임 합치기
if all_predictions:
    final_df = pd.concat(all_predictions, ignore_index=True)
    print(f"Combined dataframe has {len(final_df)} entries")

    # 열 이름 변경: 'label' -> 'rock_type'
    final_df = final_df.rename(columns={'label': 'rock_type'})

    # 최종 제출 파일 저장
    submission_path = '/content/drive/MyDrive/open/final_submission.csv'
    final_df.to_csv(submission_path, index=False)
    print(f"Final submission saved to: {submission_path}")

    # 샘플 데이터 출력
    print("\n첫 5개 예측:")
    print(final_df.head())
else:
    print("No prediction files were found or loaded.")